# 🎯 Abaque de Gy et QA/QC

Les analyses de **QA/QC** sont inspirées de :

> 📚 *Rafini, S., 2015*. **Assurance et contrôle de la qualité (QA/QC) en exploration minérale** : synthèse et évaluation des usages. Rapport, Projet CONSOREM 2013-05, 44 p.


# 🎯 Abaque de Gy interactif

Ce notebook permet de visualiser les lignes d'isocontours de l'écart-type relatif de Gy.

Vous pouvez entrer vos propres paramètres, les valider, et obtenir une procédure graphique adaptée à vos étapes.

---

## ✅ Étapes :

1. Entrer vos paramètres globaux :  
- **al** : proportion massique du lot analysé  
- **da**, **dg** : densités apparente et réelle  
- **d0** : taille du plus petit fragment (en cm)  
- **ml** : masse totale du lot (en g)  
- **s_vals** : liste des écarts-types relatifs souhaités (ex. `[0.1, 0.2]`)

2. Ajouter une ou plusieurs étapes avec leurs paramètres spécifiques :  
- **me** : masse d’échantillon (en g)  
- **ml** : masse totale à l’étape (en g)  
- **d** : taille max des fragments (en cm)

3. Visualiser l’abaque mis à jour avec :  
- les lignes d’isocontours en noir pour les écarts-types choisis  
- les points rouges représentant chaque étape saisie  
- les flèches bleues indiquant la progression entre étapes  
- le calcul et l’affichage de l’écart-type relatif global (sr global)

---

## 📊 Exemple de sortie :

L’abaque affiche une échelle logarithmique en abscisse (taille des fragments, cm) et en ordonnée (masse de l’échantillon, g).  
Les courbes noires sont les isocontours pour les écarts-types relatifs choisis, tandis que les courbes en gris pointillé correspondent à au maillage logarithmique.

Chaque point rouge correspond à une étape saisie, avec sa valeur sr affichée en rouge à côté.  
Les flèches bleues montrent le cheminement entre étapes.  

Enfin, le sr global (écart-type total combiné) est indiqué en bleu en haut à gauche.

---

> **Remarque** : les calculs reposent sur les paramètres et formules spécifiques à la géotechnique minière.  
> Assurez-vous de bien ajuster les valeurs selon votre contexte d’étude.


In [88]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch
from ipywidgets import VBox, HBox, Button, FloatText, Output, Label, Layout
from IPython.display import display, clear_output

# Variables globales initiales
params = {
    'al': 0.010 / 0.67,
    'da': 4.1,
    'dg': 2.8,
    'd0': 0.1
}

# Widgets pour les paramètres globaux
al_input = FloatText(description='al (conc.)', value=params['al'], step=0.01, layout=Layout(width='160px'))
da_input = FloatText(description='da (densité min.)', value=params['da'], step=0.1, layout=Layout(width='160px'))
dg_input = FloatText(description='dg (densité gangue)', value=params['dg'], step=0.1, layout=Layout(width='160px'))
d0_input = FloatText(description='d0 (taille lib.)', value=params['d0'], step=0.0001, layout=Layout(width='160px'))

params_widgets = HBox([al_input, da_input, dg_input, d0_input])

def gy(al, da, dg, ml, d0, me=None, d=None, sr=None, f=0.5, g=0.25):
    if d is None:
        fl = 1.0
    else:
        fl = min(np.sqrt(d0 / d), 1.0)
    ud = (1 - al) / al * ((1 - al) * da + al * dg)
    k2 = ud * f * g
    k = ud * f * g * fl
    if me is None:
        ime = sr**2 / k / d**3 + 1 / ml
        sr = 1 / ime
    elif d is None:
        for _ in range(10):
            d3 = sr**2 / k / (1 / me - 1 / ml)
            d = d3 ** (1 / 3)
            fl = min(np.sqrt(d0 / d), 1.0)
            k = ud * f * g * fl
        sr = d
    else:
        s2 = k * d**3 / me * (1 - me / ml)
        sr = np.sqrt(s2)
    return sr

def plot_gy_iso_contours(ax, al, da, dg, d0, ml, s_vals=[0.1]):
    """
    Trace les lignes d'isocontours de l'écart-type relatif de Gy sur l'axe `ax`.
    """
    f, g = 0.5, 0.25
    d = np.exp(np.linspace(-7, 3, 100))  # tailles de fragments en cm
    flib = np.minimum(1, np.sqrt(d0 / d))
    ud = (1 - al) / al * ((1 - al) * da + al * dg)
    k = ud * f * g * flib * d**3

    ymin, ymax = 1, 1e4
    for s in s_vals:
        me = 1 / (s**2 / k + 1 / ml)
        ax.loglog(d, me, 'k', linewidth=1.5)
        # Ajout texte (facultatif)
        i = np.argmin(np.abs(me - 50))
        if i < len(d):
            ax.text(d[i]*1.4, me[i], f'{s * 100:.3f}%' , fontsize=10,
                    rotation=45, color='black', ha='right')

    ax.set_xlim([5e-3, 1])
    ax.set_ylim([1, 1e4])
    ax.grid(True, which='both', linestyle='-', alpha=0.75)
    ax.set_xlabel("Taille des plus gros fragments (cm)")
    ax.set_ylabel("Masse de l'échantillon (g)")
    ax.set_title(f"Abaque de Gy – al={al}, da={da}, dg={dg}, d0={d0}, ml={ml:.0f}")
   
# Liste des widgets pour les étapes
steps = []
rows = VBox()
output = Output()
status_label = Label(value='')

def add_step(_=None):
    me_input = FloatText(description='me (g)', value=100.0, step=1.0, layout={'width': '160px'})
    ml_input = FloatText(description='ml (g)', value=1000.0, step=1.0, layout={'width': '160px'})
    d_input = FloatText(description='d (cm)', value=0.1, step=0.01, layout={'width': '160px'})
    
    steps.append((me_input, ml_input, d_input))
    row = HBox([me_input, ml_input, d_input])
    rows.children += (row,)
    
    me_input.observe(update_plot, names='value')
    ml_input.observe(update_plot, names='value')
    d_input.observe(update_plot, names='value')
    
    update_plot()

def remove_step(_=None):
    if steps:
        steps.pop()
        rows.children = rows.children[:-1]
        update_plot()

def finish_steps(_=None):
    btn_add.disabled = True
    btn_finish.disabled = True
    status_label.value = "Fin de la saisie des étapes. Plus aucune étape ne peut être ajoutée."
    update_plot()

def update_params(_=None):
    try:
        params['al'] = float(al_input.value)
        params['da'] = float(da_input.value)
        params['dg'] = float(dg_input.value)
        params['d0'] = float(d0_input.value)
    except Exception as e:
        print("Erreur de saisie dans les paramètres globaux:", e)
    update_plot()

def update_plot(_=None):
    with output:
        clear_output(wait=True)
        # Appliquer la contrainte ml_i+1 = me_i
        update_ml_from_me()

        fig, ax = plt.subplots(figsize=(8, 5))
        
        d_vals = np.logspace(-2, 1, 100)
        me_vals = np.logspace(1, 4, 100)
        D, ME = np.meshgrid(d_vals, me_vals)
        
        SR = np.zeros_like(D)
        for i in range(D.shape[0]):
            for j in range(D.shape[1]):
                SR[i, j] = gy(params['al'], params['da'], params['dg'], ME[i, j], params['d0'], me=ME[i, j], d=D[i, j])
        
        cs = ax.contour(D, ME, SR * 100, levels=[5, 10, 20, 40, 60, 80], colors='gray', linestyles='dashed')
        ax.clabel(cs, inline=True, fontsize=8, fmt='%1.0f%%')
        
        sr_list = []
        valid_coords = []
        ml_values = []
        
        for (me_input, ml_input, d_input) in steps:
            me = me_input.value
            ml = ml_input.value
            d = d_input.value
            if me > 0 and ml > 0 and d > 0:
                sr = gy(params['al'], params['da'], params['dg'], ml, params['d0'], me, d)
                if np.isnan(sr):
                    continue
                sr_list.append(sr)
                ml_values.append(ml)
                ax.plot(d, me, 'ro')
                ax.text(d * 1.1, me * 1.1, f'{sr * 100:.3f}%', fontsize=9, color='red')
                valid_coords.append((d, me))
        
        for i in range(len(valid_coords) - 1):
            x1, y1 = valid_coords[i]
            x2, y2 = valid_coords[i+1]
            ax.add_patch(FancyArrowPatch((x1, y1), (x2, y1), arrowstyle='->', color='blue', mutation_scale=12, lw=1.5))
            ax.add_patch(FancyArrowPatch((x2, y1), (x2, y2), arrowstyle='->', color='blue', mutation_scale=12, lw=1.5))
        
        ax.set_xlabel('Taille max fragments d (cm)')
        ax.set_ylabel('Masse échantillon me (g)')
        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.set_xlim(5e-3, 100)
        ax.set_ylim(10, 1e4)
        ax.grid(True, which='both')
        ax.set_title('Abaque de Gy - Écarts-types relatifs par étape')
        
        sr_global = None
        if sr_list:
            sr_global = np.sqrt(np.sum(np.array(sr_list) ** 2))
            ax.text(0.02, 0.9 * ax.get_ylim()[1], f"sr global = {sr_global * 100:.3f}%", color='blue', fontsize=12)
        
        ml_moyen = np.max(ml_values) if ml_values else 1000
        
        plot_gy_iso_contours(ax, params['al'], params['da'], params['dg'], params['d0'],
                             ml=ml_moyen, s_vals=[0.001, 0.005, 0.01, 0.05, 0.1, 0.2])
        
        plt.show()

        # Affichage des valeurs Sr par étape dans la sortie texte
        if sr_list:
            print("\nValeurs des Sr par étape :")
            print("(Ligne 1 = 1er échantillon, Ligne 2 = 2ème, etc.)\n")
            for i, sr_val in enumerate(sr_list, 1):
                print(f"Sr{i} = {sr_val*100:.3f} %")
            print(f"\nSr global = {sr_global*100:.3f} %")
        
        # Validation du sr global si défini
        if sr_global is not None:
            sr_desire = sr_desire_input.value / 100
            if sr_global <= sr_desire:
                validation_label.value = f"✅ Procédure valide (sr_global = {sr_global*100:.3f}% ≤ sr désiré = {sr_desire*100:.3f}%)"
                validation_label.color = 'green'
            else:
                validation_label.value = f"❌ Procédure NON valide (sr_global = {sr_global*100:.3f}% > sr désiré = {sr_desire*100:.3f}%)"
                validation_label.color = 'red'
        else:
            validation_label.value = ""

btn_add = Button(description='Ajouter étape', button_style='success')
btn_add.on_click(add_step)

btn_remove = Button(description='Supprimer dernière étape', button_style='warning')
btn_remove.on_click(remove_step)

btn_finish = Button(description='Fin', button_style='danger')
btn_finish.on_click(finish_steps)

al_input.observe(update_params, names='value')
da_input.observe(update_params, names='value')
dg_input.observe(update_params, names='value')
d0_input.observe(update_params, names='value')


ui = VBox([
    Label("Paramètres globaux :"), 
    params_widgets,
    HBox([Label("sr désiré (%) :"), sr_desire_input]),
    HBox([btn_add, btn_remove, btn_finish]), 
    rows, 
    status_label,
    validation_label,
    output
])

display(ui)



## 🧪 Analyse des blancs – Visualisation et interprétation

Ce graphique représente une série temporelle de mesures de blancs analytiques, c’est-à-dire des échantillons censés ne contenir aucun élément détectable.  
Ces mesures sont utilisées pour vérifier la qualité des analyses et détecter d’éventuelles contaminations.

### 🎯 Objectifs de l’exercice :

- Visualiser les blancs dans l’ordre d’analyse.
- Contrôler la variabilité des mesures à l’aide de bandes d’erreur statistiques (±1LD, ±2LD, ±3LD). LD est la limite de détection de l'appareil, il est courant de basé la mesure d'erreur sur cette valeur. Des fois, on se basse sur un pourcentage de la teneur de coupure, p.ex, 5% t.c..
- Identifier les valeurs aberrantes (hors de la distribution normale attendue).
- Comparer les mesures aux seuils d’acceptabilité :
  - Seuils basés sur la limite de détection (LD) : 3LD, 5LD, 10LD.
  - Seuils relatifs à la teneur de coupure : 5% et 10%.

### 📊 Ce que montre le graphique :

- Les points verts représentent les mesures des blancs.
- Les bandes vertes autour de la ligne de base (0 ppm) indiquent les intervalles de confiance statistiques.
- Les lignes bleues sont les seuils de contrôle utilisés pour détecter les échecs.
- Les points rouges signalent les valeurs aberrantes.
- Le nombre de valeurs dépassant la limite de détection est affiché dans le titre.


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

def generate_blank_series(n_points=1000, noise_level=1.0):
    # Le bruit des blancs est généré par une gaussien tronqué, c'est`à dire que les valeurs inférieures à 0 sont amenées à 0.
    t = np.arange(n_points)
    base = 0.0
    noise = np.random.normal(0, noise_level, size=n_points)
    series = base + noise
    series = np.clip(series, 0, None)  # Pas de teneur négative
    return t, series

def plot_blank_series(noise_level=1, standard_error=1):
    np.random.seed(42)
    n_points = 1000
    t, blancs = generate_blank_series(n_points=n_points, noise_level=noise_level)

    plt.figure(figsize=(12, 5))

    # Tracé des blancs
    plt.plot(t, blancs, label="Blancs mesurés", color='green', linestyle='', marker='x')

    base_blanc = np.zeros_like(t)

    # Zones ±3LD, ±5LD, ±10LD
    limits = [3, 5, 10]
    alphas = [0.3, 0.2, 0.1]

    for k, alpha in zip(limits, alphas):
        lower_bound = np.maximum(base_blanc - k * standard_error, 0)  # Ne pas descendre sous 0
        upper_bound = base_blanc + k * standard_error
        plt.fill_between(t, lower_bound, upper_bound,
                         color='green', alpha=alpha, label=f'+{k} LD')

    plt.plot(t, base_blanc, color='green', linestyle='--', label="Teneur attendue (blanc)")

    # Détection des outliers
    diff = blancs - base_blanc  # Comme base = 0, c'est juste blancs
    circle_sizes = 50

    # Préparer couleurs vides et on remplit selon seuils
    colors = np.array(['none'] * len(diff))

    # Seuils pour outliers
    idx_3LD = diff > 10 * standard_error
    idx_2LD = (diff > 5 * standard_error) & (~idx_3LD)
    idx_1LD = (diff > 3 * standard_error) & (~idx_2LD) & (~idx_3LD)

    # Assignation des couleurs
    colors[idx_1LD] = '#ff4d4d'  # rouge clair vif
    colors[idx_2LD] = '#cc0000'  # rouge moyen foncé
    colors[idx_3LD] = '#660000'  # rouge très foncé

    # Filtrer pour ne garder que les outliers
    outlier_indices = np.where(colors != 'none')[0]

    if outlier_indices.size > 0:
        plt.scatter(t[outlier_indices], blancs[outlier_indices],
                    s=circle_sizes,
                    color=colors[outlier_indices],
                    label="Outliers détectés",
                    edgecolors='black', linewidths=0.7,
                    alpha=0.9)

    # Compter les outliers
    count_1LD = np.sum(idx_1LD)
    count_2LD = np.sum(idx_2LD)
    count_3LD = np.sum(idx_3LD)
    total_outliers = count_1LD + count_2LD + count_3LD

    # Ajouter le compteur au titre
    plt.title(f"Série temporelle des blancs avec limites de détection et détection automatique des outliers\n"
              f"Total outliers: {total_outliers} (1LD: {count_1LD}, 2LD: {count_2LD}, 3LD: {count_3LD})")

    plt.xlabel("Temps (échantillon)")
    plt.ylabel("Teneur (ppm)")
    plt.ylim(-0.5, 30 * standard_error)  # Axe y fixé à 30 fois LD
    plt.legend()
    plt.grid(True)
    plt.show()

interact(
    plot_blank_series,
    noise_level=FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description="Niveau de bruit"),
    standard_error=FloatSlider(value=0.5, min=0.1, max=5.0, step=0.1, description="Limite de détection (LD)"),
)

interactive(children=(FloatSlider(value=1.0, description='Niveau de bruit', max=5.0, min=0.1), FloatSlider(val…

<function __main__.plot_blank_series(noise_level=1, standard_error=1)>

# 🔬 Contrôle Qualité des Séries de Standards en Laboratoire

---

## 📚 Introduction

Dans ce notebook, nous explorons des concepts clés du **contrôle qualité** appliqué aux mesures répétées de standards analytiques, notamment :

- 🧪 **Génération simulée** de séries temporelles de mesures standards, intégrant différents types d’anomalies fréquentes en laboratoire (erreurs de transcription, changements de méthode, tendances).
- 🔍 **Détection automatique d’anomalies** à partir des règles statistiques classiques basées sur les écarts types (±1σ, ±2σ, ±3σ) et leur interprétation.
- 📊 **Visualisation interactive** permettant d’explorer l’impact des différents paramètres (niveau de bruit, nombre et amplitude des erreurs, changement de méthode) sur la qualité des mesures et la robustesse des contrôles statistiques.

---

## 🎯 Objectifs pédagogiques

À la fin de cette séance, vous serez capable de :

- ⚠️ Comprendre les sources potentielles d’anomalies dans une série de mesures répétées d’un standard.
- 📏 Appliquer des règles de contrôle statistique pour identifier ces anomalies.
- 📈 Interpréter graphiquement les résultats de la détection d’anomalies.
- 🕹️ Utiliser des widgets interactifs pour simuler différents scénarios et mieux appréhender la variabilité naturelle et les déviations anormales.

---

## 🔍 Détection automatique des anomalies (règles de Western Electric)

Basée sur la moyenne (μ) et l’écart-type (σ), 4 règles empiriques détectent les signaux d’un processus potentiellement hors de contrôle :

**Liste des critères**
1. **Un point au-delà de ±3σ**  
   \(|x - \mu| > 3\sigma\) → Anomalie majeure  
   Probabilité ≈ 0.27 % — signal fort d’un événement exceptionnel.

2. **Deux points consécutifs au-delà de ±2σ, du même côté**  
   \(x_1, x_2 > \mu + 2\sigma\) ou \(< \mu - 2\sigma\) → Biais temporaire suspecté  
   Seuil d’alerte ≈ 1 % (valeur empirique).

3. **Quatre points consécutifs au-delà de ±1σ, du même côté**  
   \(x_1, ..., x_4 > \mu + \sigma\) ou \(< \mu - \sigma\) → Dérive progressive  
   Seuil d’alerte ≈ 1 % (valeur empirique).

4. **Huit points consécutifs du même côté de la moyenne (μ)**  
   \(x_1, ..., x_8 > \mu\) ou \(< \mu\) → Changement systématique  
   Seuil d’alerte ≈ 1 % (valeur empirique).

---

> 🧠 *Ces seuils sont empiriques, choisis pour un bon compromis entre détection d’anomalies et faux positifs, et peuvent différer des calculs théoriques sous hypothèses normale.*





In [91]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider, Checkbox

def generate_standard_series(
    n_points=500,
    noise_level=1.0,
    trend_slope=0.0,
    n_transcription_errors=0,
    transcription_error_magnitude=2.0,
    method_change=False,
    method_change_point=250,
    method_change_magnitude=5.0,
    error_zone_fraction=0.2
):
    t = np.arange(n_points)
    base = 50 + trend_slope * (t - round(len(t)/2))
    noise = np.random.normal(0, noise_level, size=n_points)
    series = base + noise

    max_index_for_errors = int(n_points * error_zone_fraction)

    if n_transcription_errors > 0 and max_index_for_errors > 0:
        indices = np.random.choice(max_index_for_errors, size=n_transcription_errors, replace=False)
        errors = np.random.choice([-1, 1], size=n_transcription_errors) * transcription_error_magnitude
        series[indices] += errors

    if method_change:
        series[method_change_point:] += method_change_magnitude

    return t, series

def detect_anomalies(series, mean, std):
    n = len(series)
    anomalies = {
        "Critère 1": [],
        "Critère 2": [],
        "Critère 3": [],
        "Critère 4": [],
    }

    for i in range(n):
        if abs(series[i] - mean) > 3 * std:
            anomalies["Critère 1"].append(i)

    for i in range(n - 1):
        cond1 = (series[i] - mean) > 2 * std and (series[i+1] - mean) > 2 * std
        cond2 = (series[i] - mean) < -2 * std and (series[i+1] - mean) < -2 * std
        if cond1 or cond2:
            anomalies["Critère 2"].extend([i, i+1])

    side = np.sign(series - mean)
    outside_1sigma = np.abs(series - mean) > std
    count = 0
    for i in range(n):
        if outside_1sigma[i]:
            if i == 0 or (side[i] == side[i-1] and side[i] != 0):
                count += 1
            else:
                count = 1
        else:
            count = 0
        if count >= 4:
            anomalies["Critère 3"].append(i)

    count_8 = 0
    for i in range(n):
        if i == 0 or (side[i] == side[i-1] and side[i] != 0):
            count_8 += 1
        else:
            count_8 = 1
        if count_8 >= 8:
            anomalies["Critère 4"].append(i)

    return anomalies

def plot_standard_series(
    error_zone_fraction,
    noise_level,
    standard_error,
    trend_slope,
    n_transcription_errors,
    transcription_error_magnitude,
    method_change,
    method_change_point,
    method_change_magnitude,
):
    np.random.seed(42)
    t, series = generate_standard_series(
        noise_level=noise_level,
        trend_slope=trend_slope,
        n_transcription_errors=n_transcription_errors,
        transcription_error_magnitude=transcription_error_magnitude,
        method_change=method_change,
        method_change_point=method_change_point,
        method_change_magnitude=method_change_magnitude,
        error_zone_fraction=error_zone_fraction,
    )

    base_line = 50 + 0 * t
    anomalies = detect_anomalies(series, mean=50.0, std=standard_error)

    plt.figure(figsize=(12, 5))
    plt.plot(t, series, label="Standard mesuré", color='blue', linestyle='', marker='*')

    colors = ['#ffcc80', '#ffb74d', '#ffa726']
    alphas = [0.3, 0.2, 0.1]
    for k, alpha, color in zip([1, 2, 3], alphas, colors):
        plt.fill_between(
            t,
            base_line - k * standard_error,
            base_line + k * standard_error,
            color=color,
            alpha=alpha,
            label=f'±{k}σ',
            edgecolor=None,
        )

    plt.plot(t, base_line, color='orange', linestyle='--', label="Teneur attendue (50 ppm)")

    markers_info = {
        "Critère 1": ("red", 80, 'o'),
        "Critère 2": ("purple", 60, 's'),
        "Critère 3": ("brown", 50, '^'),
        "Critère 4": ("green", 40, 'D'),
    }

    for crit, (color, size, marker) in markers_info.items():
        indices = list(set(anomalies[crit]))
        plt.scatter(t[indices], series[indices], color=color, label=crit, s=size, marker=marker, edgecolors='k', zorder=5)

    if method_change:
        plt.axvline(method_change_point, color="red", linestyle="--", label="Changement méthode")

    plt.xlabel("Temps (échantillon)")
    plt.ylabel("Teneur standard (ppm)")
    plt.title("Série temporelle de standards avec détection d’anomalies (±σ)")
    plt.ylim(42, 58)

    # Légende à l'extérieur à droite
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=9)
    plt.grid(True)
    plt.tight_layout(rect=[0, 0, 0.85, 1])  # laisse la place pour la légende
    plt.show()

interact(
    plot_standard_series,
    error_zone_fraction=FloatSlider(value=0.2, min=0.05, max=0.5, step=0.05, description="Zone erreurs début"),
    noise_level=FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description="Niveau bruit"),
    standard_error=FloatSlider(value=1.0, min=0.5, max=2, step=0.1, description="Erreur type"),
    trend_slope=FloatSlider(value=0.0, min=-0.01, max=0.01, step=0.001, description="Tendance"),
    n_transcription_errors=IntSlider(value=0, min=0, max=20, step=1, description="Erreurs transcription"),
    transcription_error_magnitude=FloatSlider(value=2, min=1, max=4, step=0.01, description="Amplitude erreur"),
    method_change=Checkbox(value=False, description="Changement méthode"),
    method_change_point=IntSlider(value=250, min=1, max=499, step=1, description="Point changement"),
    method_change_magnitude=FloatSlider(value=0.0, min=-1, max=1, step=0.1, description="Amplitude changement"),
)




interactive(children=(FloatSlider(value=0.2, description='Zone erreurs début', max=0.5, min=0.05, step=0.05), …

<function __main__.plot_standard_series(error_zone_fraction, noise_level, standard_error, trend_slope, n_transcription_errors, transcription_error_magnitude, method_change, method_change_point, method_change_magnitude)>

## Simulation de QA/QC sur duplicatas géochimiques avec incertitude contrôlée

Cette section présente trois graphiques types couramment utilisés dans l’évaluation des duplicatas en contrôle de la qualité (QA/QC).  
Nous explorerons ensemble leurs points forts et leurs limites, en mettant en évidence ce qu’ils permettent de détecter — ou non — dans les écarts entre duplicatas.

### 📊 1. Nuage de points des duplicatas

Ce graphique compare directement les deux séries simulées de duplicatas.  
- La ligne noire en pointillé représente l’égalité parfaite (Duplicata 1 = Duplicata 2).
- Les bandes colorées indiquent les tolérances acceptables de ±10%, ±20% et ±30%.
- Les points rouges indiquent les cas où l’écart dépasse ±10%, signalant un problème potentiel de reproductibilité.

Cela permet une évaluation visuelle immédiate du respect des critères QA/QC selon les tolérances définies. Cependant, il n'est pas très informatif. On a recourd généralement aux graphiques des points 2 et 3.

### 📈 2. Différence relative (%) selon la moyenne des duplicatas
Ce graphique montre la différence relative entre les deux duplicatas en pourcentage, en fonction de leur moyenne :
- Il met en évidence les écarts systématiques ou aléatoires.
- Les lignes pointillées indiquent les niveaux de tolérance.
- Les points rouges signalent les duplicatas hors tolérance de ±10 %.

Ce graphique permet d’identifier si les écarts entre duplicatas sont constants ou proportionnels à l’intensité des valeurs — un effet souvent appelé *effet multiplicatif*.  
On observe généralement que les faibles teneurs présentent des erreurs relatives plus élevées que les fortes teneurs.  
Il est donc essentiel de porter une attention particulière à la zone autour de la teneur de coupure, où ces erreurs peuvent avoir un impact significatif sur les décisions d’exploitation.


### 📐 3. Courbe HARD (Half Absolute Relative Difference)

Le graphique HARD trace la courbe cumulative de l’erreur relative :
- Sur l’axe vertical, on mesure l’écart relatif (|D1 − D2| / (D1 + D2)).
- L’axe horizontal correspond au rang normalisé des points (i.e., leur position dans la distribution triée).
- Le point rouge représente un objectif typique (par ex. 90% des duplicatas dans ±10%).

Ce graphique est souvent utilisé pour évaluer la **performance globale du protocole de QA/QC**.


In [90]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

def generate_correlated_lognormal_series_mv_variable_noise(
    n_points=100,
    base_median=2,
    sigma_base=0.4,
    correlation=0.95,
    p=0.0,  # nouveau paramètre contrôle la pente du sigma selon la valeur
):
    mu = np.log(base_median)

    # Tirage base_vals avec sigma fixe non nul (exemple 0.2)
    base_vals = np.exp(np.random.normal(mu, 0.2, size=n_points))

    safe_vals = np.clip(base_vals, 1e-3, None)
    sigma_vals = sigma_base  # scalaire

    corr_mat = np.array([[1.0, correlation],
                         [correlation, 1.0]])

    dup1 = np.empty(n_points)
    dup2 = np.empty(n_points)
    for i in range(n_points):
        cov = corr_mat * sigma_vals**2  # <-- ici on utilise sigma_vals comme scalaire
        noise = np.random.multivariate_normal(mean=[0, 0], cov=cov)
        dup1[i] = np.exp(mu + noise[0])
        dup2[i] = np.exp(mu + noise[1])

    dup1 += np.random.normal(0, p, size=n_points)
    dup2 += np.random.normal(0, p, size=n_points)

    return dup1, dup2


def plot_lognormal_variable_noise(median=2.0, sigma=0.4, corr=0.9, p=0.0):
    np.random.seed(42)

    dup1, dup2 = generate_correlated_lognormal_series_mv_variable_noise(
        n_points=200,
        base_median=median,
        sigma_base=sigma,
        correlation=corr,
        p=p,
    )

    fig = plt.figure(figsize=(18, 6), constrained_layout=True)

    max_val = min(max(np.quantile(dup1, 0.95), np.quantile(dup2, 0.95)) * 1.1, 100)

    # --- Scatter duplicatas ---
    ax1 = fig.add_subplot(1, 3, 1)
    ax1.scatter(dup1, dup2, alpha=0.6, label="Points")

    lims = [0, max_val]
    ax1.plot(lims, lims, 'k--', label="y = x")

    tolerances = [0.1, 0.2, 0.3]
    colors = ['r', 'orange', 'purple']

    counts_out = []
    for tol, col in zip(tolerances, colors):
        lower = 1 - tol
        upper = 1 + tol
        ax1.plot(lims, [lims[0]*upper, lims[1]*upper], color=col, linestyle='-', alpha=0.6, label=f"±{int(tol*100)}%")
        ax1.plot(lims, [lims[0]*lower, lims[1]*lower], color=col, linestyle='-', alpha=0.6)
        out_of_bounds = (dup2 < dup1 * lower) | (dup2 > dup1 * upper)
        counts_out.append(np.sum(out_of_bounds))

    tol_max = 0.1
    lower_max = 1 - tol_max
    upper_max = 1 + tol_max
    out_max = (dup2 < dup1 * lower_max) | (dup2 > dup1 * upper_max)
    ax1.scatter(dup1[out_max], dup2[out_max], color='red', s=80, label='Hors ±10%')

    median_r = round(median, 2)
    sigma_r = round(sigma, 2)
    corr_r = round(corr, 2)
    p_r = round(p, 2)

    ax1.set_xlabel("Duplicata 1")
    ax1.set_ylabel("Duplicata 2")
    ax1.set_title(f"Séries lognormales corrélées\nMédiane={median_r}, Sigma={sigma_r}, Corr={corr_r}, p={p_r}\n"
              f"Hors ±10%: {counts_out[0]} | ±20%: {counts_out[1]} | ±30%: {counts_out[2]} sur {len(dup1)} points")
    ax1.grid(True)
    ax1.legend(loc='best')
    ax1.set_xlim(0, max_val)
    ax1.set_ylim(0, max_val)

    # --- Différence relative ---
    ax2 = fig.add_subplot(1, 3, 2)
    mean_vals = (dup1 + dup2) / 2
    diff_rel = 100 * (dup1 - dup2) / mean_vals

    ax2.scatter(mean_vals, diff_rel, alpha=0.6, color='blue', label="Différence relative (%)")

    for tol, col in zip(tolerances, colors):
        ax2.axhline(y=tol*100, color=col, linestyle='--', alpha=0.6, label=f'±{int(tol*100)}%')
        ax2.axhline(y=-tol*100, color=col, linestyle='--', alpha=0.6)

    out_diff_max = (diff_rel > 10) | (diff_rel < -10)
    ax2.scatter(mean_vals[out_diff_max], diff_rel[out_diff_max], color='red', s=80, label='Hors ±10%')

    ax2.set_xlabel("(Duplicata 1 + Duplicata 2) / 2")
    ax2.set_ylabel("Différence relative (%)")
    ax2.set_title("Différence relative entre duplicatas")
    ax2.grid(True)
    ax2.legend(loc='best')
    ax2.set_xlim(0, max_val)
    ax2.set_ylim(-50, 50)

    # --- HARD plot ---
    ax3 = fig.add_subplot(1, 3, 3)
    N = len(dup1)
    hard_vals = np.sort(np.abs(dup1 - dup2) / (dup1 + dup2))
    ranks = np.arange(1, N + 1) / (N + 1)

    ax3.plot(ranks, hard_vals, color='black', linewidth=2, label='Cible')
    ax3.plot(0.9, 0.1, 'o', color='red', markersize=10, label='Point critique')
    ax3.set_xlabel('Rang/(N+1)')
    ax3.set_ylabel('Graphique HARD')
    ax3.set_ylim(0, max(0.3, hard_vals.max() * 1.1))  # plus flexible
    ax3.grid(True)
    ax3.legend(loc='best')

    plt.show()


interact(
    plot_lognormal_variable_noise,
    median=FloatSlider(min=0.1, max=10, step=0.001, value=0.9, description="Médiane"),
    sigma=FloatSlider(min=0.05, max=1.0, step=0.001, value=1.4, description="Sigma base"),
    corr=FloatSlider(min=0.95, max=0.999, step=0.001, value=0.996, description="Corrélation", readout_format=".3f"),
    p=FloatSlider(min=0, max=2.0, step=0.05, value=0, description="Bruit décroissant p")
)




interactive(children=(FloatSlider(value=0.9, description='Médiane', max=10.0, min=0.1, step=0.001), FloatSlide…

<function __main__.plot_lognormal_variable_noise(median=2.0, sigma=0.4, corr=0.9, p=0.0)>